In [1]:
pip install langchain_openai langchain langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import numpy as np
import json
import os
from scipy.interpolate import interp1d
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def fetch_windborne_data(hours=24):
    all_data = {}
    for h in range(hours):
        url = f"https://a.windbornesystems.com/treasure/{h:02d}.json"
        response = requests.get(url)
        if response.status_code == 200:
            try:
                data = response.json()
                if isinstance(data, list) and all(isinstance(item, list) and len(item) == 3 for item in data):
                    all_data[h] = np.array(data)
                else:
                    print(f"Warning: Skipping malformed data from {url}")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from {url}: {e}")
        else:
            print(f"Failed to fetch {url}, status code: {response.status_code}")

    if not all_data:
        print("No valid data collected.")
        return None
    
    # Ensure data is structured as (hours, balloons, coordinates) only for existing hours
    num_balloons = max(arr.shape[0] for arr in all_data.values())
    structured_data = np.full((len(all_data), num_balloons, 3), np.nan)
    
    for i, (h, data) in enumerate(all_data.items()):
        structured_data[i, :data.shape[0], :] = data
    
    # Fill missing values with column mean
    for d in range(3):  # Iterate over x, y, z coordinates
        valid_mask = ~np.isnan(structured_data[:, :, d])
        if valid_mask.any():  # If at least one valid value exists
            mean_value = np.nanmean(structured_data[:, :, d])
            structured_data[:, :, d] = np.where(np.isnan(structured_data[:, :, d]), mean_value, structured_data[:, :, d])
    
    return structured_data

all_positions = fetch_windborne_data()

if all_positions is not None:
    print("Data Shape (after full interpolation):", all_positions.shape)
    print("Any NaN remaining?", np.isnan(all_positions).any())
else:
    print("No valid data available.")

Failed to fetch https://a.windbornesystems.com/treasure/01.json, status code: 404


Failed to fetch https://a.windbornesystems.com/treasure/05.json, status code: 404


Failed to fetch https://a.windbornesystems.com/treasure/06.json, status code: 404


Failed to fetch https://a.windbornesystems.com/treasure/08.json, status code: 404


Failed to fetch https://a.windbornesystems.com/treasure/09.json, status code: 404


...

Failed to fetch https://a.windbornesystems.com/treasure/19.json, status code: 404


Error decoding JSON from https://a.windbornesystems.com/treasure/20.json: Extra data: line 6 column 6 (char 96)


Failed to fetch https://a.windbornesystems.com/treasure/21.json, status code: 404


Failed to fetch https://a.windbornesystems.com/treasure/22.json, status code: 404


Failed to fetch https://a.windbornesystems.com/treasure/23.json, status code: 404
Data Shape (after full interpolation): (10, 1000, 3)
Any NaN remaining? False


In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score

def process_sequences(data, time_steps=5):
    """Prepare LSTM sequences for training, ensuring correct order."""
    num_balloons = data.shape[1]
    X, y = [], []
    scalers = [MinMaxScaler() for _ in range(num_balloons)]

    for i in range(num_balloons):
        balloon_data = data[:, i, :][::-1]  # Reverse order: [08, 07, ..., 00]
        balloon_data = scalers[i].fit_transform(balloon_data)

        # Generate time-step sequences
        for j in range(len(balloon_data) - time_steps):
            X.append(balloon_data[j:j+time_steps])
            y.append(balloon_data[j+time_steps])

    return np.array(X), np.array(y), scalers

# Prepare LSTM sequences
time_steps = 5
X, y, scalers = process_sequences(all_positions, time_steps)
print("LSTM Input Shape:", X.shape)  # Expected: (num_samples, time_steps, 3)

# Define Early Stopping
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Define LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, activation="relu", input_shape=(time_steps, 3)),
    LSTM(32, return_sequences=False, activation="relu"),
    Dense(16, activation="relu"),
    Dense(3)  # Predicts (lat, lon, alt)
])

model.compile(optimizer=Adam(learning_rate=0.001, clipnorm=1.0), loss="mse")

# Train Model
history = model.fit(X, y, epochs=30, batch_size=16, validation_split=0.1, callbacks=[early_stop])

y_pred = model.predict(X)
mse = mean_squared_error(y.reshape(-1, 3), y_pred.reshape(-1, 3))
r2 = r2_score(y.reshape(-1, 3), y_pred.reshape(-1, 3))

print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"R² Score: {r2:.6f}")

def predict_next_position(model, all_positions, scalers, time_steps=5):
    """Predict the next step using the latest available data from each balloon."""
    num_balloons = len(scalers)
    predicted_next_real = []

    for i in range(num_balloons):
        balloon_data = all_positions[:, i, :][::-1]  # Reverse to get the latest first
        balloon_data_scaled = scalers[i].transform(balloon_data)
        latest_sequence = balloon_data_scaled[:time_steps].reshape(1, time_steps, 3)

        predicted_scaled = model.predict(latest_sequence, verbose=0)
        predicted_real = scalers[i].inverse_transform(predicted_scaled)
        predicted_next_real.append(predicted_real)

    return np.array(predicted_next_real)

# Predict the next time step
predicted_next_position = predict_next_position(model, all_positions, scalers, time_steps)

print("Predicted Next Positions (Lat, Lon, Alt):")
print(predicted_next_position)


2025-03-05 23:54:03.941892: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 23:54:03.945100: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 23:54:03.953785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741218843.967865    1951 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741218843.972031    1951 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 23:54:03.988243: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

LSTM Input Shape: (5000, 5, 3)
Epoch 1/30


2025-03-05 23:54:05.949007: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


  1/282 ━━━━━━━━━━━━━━━━━━━━ 9:35 2s/step - loss: 0.3891

 20/282 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3918 

...

126/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Mean Squared Error (MSE): 0.013386
R² Score: 0.900913


Predicted Next Positions (Lat, Lon, Alt):
[[[ -63.69074    -143.81465      20.65052   ]]

 [[ -37.38792     -46.419662      6.2457356 ]]

 [[   9.946086    -29.982048      9.791365  ]]

 ...

 [[   8.546957   -106.35255      20.42935   ]]

 [[  65.625465      0.20433088    6.203978  ]]

 [[ -20.072727     97.26557      13.531021  ]]]


In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("❌ OPENAI_API_KEY is not set. Check your GitHub Secrets.")

template = """You are an operational analyst for a weather balloon company. Analyze the data of 
balloon positions over 24H: {all_positions}, LSTM prediction: {LSTM prediction}, pred next position: {pred next position}

Extract 3 key insights, such as:
- Spatial clusters indicating wind patterns
- Anomalies (e.g., balloons stuck in one area)
- Suggestions for optimizing future launches
Format the response as a bullet-point report."""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

# Generate insights
chain = prompt | model
report = chain.invoke({"all_positions": str(all_positions), "LSTM prediction": str(y_pred), "pred next position": str(predicted_next_position)})
print(report.content)

- Spatial clusters indicating wind patterns: By analyzing the balloon positions over 24 hours, we can identify spatial clusters of balloons that indicate consistent wind patterns in certain regions. This information can be valuable for predicting and planning future balloon launches, taking into account the expected wind patterns at different altitudes.

- Anomalies: There are no clear anomalies observed in the data, as the balloon positions seem to be distributed evenly across different regions. However, we should continue monitoring the data for any potential anomalies, such as balloons getting stuck in one area or unusual movement patterns.

- Suggestions for optimizing future launches: Based on the LSTM prediction of the next position for each balloon, we can optimize future launches by adjusting the launch timing and trajectory to ensure the balloons reach their desired positions efficiently. Additionally, we can consider incorporating real-time weather data and wind predictions t

In [5]:
insights_text = report.content if isinstance(report.content, str) else str(report.content)

html_output = f"""
<html><head><title>Windborne Balloon Analysis</title></head>
<body>
    <h1>Windborne Balloon Analysis</h1>
    <p><strong>Mean Squared Error (MSE):</strong> {mse:.6f}</p>
    <p><strong>R² Score:</strong> {r2:.6f}</p>
    <h2>Predicted Next Position:</h2>
    <pre>{predicted_next_position.tolist()}</pre>
    <h2>Insights:</h2>
    <pre>{insights_text}</pre>  <!-- Ensures correct display -->
</body></html>
"""

# Write HTML file
with open("docs/index.html", "w", encoding="utf-8") as f:
    f.write(html_output)

print("✅ Output saved in docs/index.html")


✅ Output saved in docs/index.html
